In [84]:
# import sys

import numpy as np
from warnings import warn

from aif360.datasets import GermanDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from common_utils import compute_metrics
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_german
from sklearn.preprocessing import MaxAbsScaler
from aif360.algorithms.inprocessing import MetaFairClassifier
from aif360.algorithms.postprocessing import RejectOptionClassification


from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display

In [39]:
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]
dataset_orig = load_preproc_data_german(['age'])

In [52]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig.favorable_label, dataset_orig.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig.privileged_protected_attributes, 
      dataset_orig.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig.feature_names)

#### Training Dataset shape

(1000, 11)


#### Favorable and unfavorable labels

1.0 2.0


#### Protected attribute names

['age']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['age', 'sex', 'credit_history=Delay', 'credit_history=None/Paid', 'credit_history=Other', 'savings=500+', 'savings=<500', 'savings=Unknown/None', 'employment=1-4 years', 'employment=4+ years', 'employment=Unemployed']


In [88]:
# split data into epochs, each with a different group of agents
NUM_EPOCHS = 1
dataset_orig_epochs = dataset_orig.split(NUM_EPOCHS, shuffle=True)

In [89]:
for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")

    dataset_epoch = dataset_orig_epochs[epoch]

    # split each data epoch into train and test
    dataset_orig_train, dataset_orig_vt = dataset_epoch.split([0.7], shuffle=True)
    dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)
     
    # print out some labels, names, etc.
    display(Markdown("#### Training Dataset shape"))
    print(dataset_orig_train.features.shape)
    print(dataset_orig_valid.features.shape)
    
    # Logistic regression classifier and predictions
    scale_orig = StandardScaler()

    # Train
    X_train = scale_orig.fit_transform(dataset_orig_train.features)
    y_train = dataset_orig_train.labels.ravel()
    lmod = LogisticRegression(solver='liblinear')  # Solver specified to avoid future warnings
    lmod.fit(X_train, y_train)

    # Predict training data
    y_train_pred = lmod.predict(X_train)
    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_train_pred.labels = y_train_pred

    # indices of favorable label
    pos_ind = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]

    # VALIDATION SET
    dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
    X_valid = scale_orig.transform(dataset_orig_valid_pred.features)
    y_valid = dataset_orig_valid_pred.labels
    dataset_orig_valid_pred.scores = lmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

    # Predict test data
    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = scale_orig.transform(dataset_orig_test_pred.features)
    y_test_pred = lmod.predict_proba(X_test)
    dataset_orig_test_pred.scores = y_test_pred[:, pos_ind].reshape(-1, 1)

    # Evaluate fairness metrics
    metric_train = BinaryLabelDatasetMetric(dataset_orig_train_pred, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
    metric_test = BinaryLabelDatasetMetric(dataset_orig_test_pred, 
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)

    print("Training set: Difference in mean outcomes = {:.3f}".format(metric_train.mean_difference()))
    print("Test set: Difference in mean outcomes = {:.3f}".format(metric_test.mean_difference()))

    # NO FAIRNESS: find best classification threshold

    num_thresh = 100
    ba_arr = np.zeros(num_thresh)
    class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
    for idx, class_thresh in enumerate(class_thresh_arr):
        
        fav_inds = dataset_orig_valid_pred.scores > class_thresh
        dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
        dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label
        
        classified_metric_orig_test = ClassificationMetric(dataset_orig_test,
                                                dataset_orig_test_pred, 
                                                unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        
        ba_arr[idx] = 0.5*(classified_metric_orig_test.true_positive_rate()\
                        +classified_metric_orig_test.true_negative_rate())

    best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
    best_class_thresh = class_thresh_arr[best_ind]

    display(Markdown("### Original Classification"))
    print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
    print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)

    # Metrics for the test set
    fav_inds = dataset_orig_test_pred.scores > best_class_thresh
    dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
    dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

    display(Markdown("#### Test set"))
    display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                    unprivileged_groups, privileged_groups)
    

    ###### USING REJECT OPTION CLASSIFICATION

    # Metric used (should be one of allowed_metrics)
    metric_name = "Statistical parity difference"

    # Upper and lower bound on the fairness metric used
    metric_ub = 0.05
    metric_lb = -0.05

    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)

    display(Markdown("### Reject Option Classificatoin"))
    print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # Metrics for the transformed test set
    dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

    display(Markdown("#### Test set"))
    display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                unprivileged_groups, privileged_groups)
    
    


Epoch 1/1


#### Training Dataset shape

(700, 11)
(150, 11)
Training set: Difference in mean outcomes = -0.001
Test set: Difference in mean outcomes = -0.328


### Original Classification

Best balanced accuracy (no fairness constraints) = 0.5442
Optimal classification threshold (no fairness constraints) = 0.6534


...

### Reject Option Classificatoin

Optimal classification threshold (with fairness constraints) = 0.6930
Optimal ROC margin = 0.0752


#### Test set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.6591
Statistical parity difference = -0.2169
Disparate impact = 0.5286
Average odds difference = -0.1167
Equal opportunity difference = -0.1751
Theil index = 0.4623
